In [1]:
import joblib
import pandas as pd
import numpy as np
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

%matplotlib inline

tweets = pd.read_csv('../tweets/tweets_clean.csv',
                     header=0,
                     parse_dates=['date'])
tweets.dropna(subset=['lemmas'], inplace=True)

# Load vectorizer
with open('../scripts/topic_modeling_objects/sklearn_vect.joblib', 'rb') as f:
    cv = joblib.load(f)

# Load term frequency matrix
with open('../scripts/topic_modeling_objects/sklearn_CV.joblib', 'rb') as f:
    tf = joblib.load(f)

# Load feature names
with open('../scripts/topic_modeling_objects/sklearn_feature_names.joblib', 'rb') as f:
    tf_names = joblib.load(f)

# Load fitted LDA model
with open('../scripts/topic_modeling_objects/sklearn_LDA_model.joblib', 'rb') as f:
    lda_model = joblib.load(f)


In [2]:
tweets_orig = tweets.text.tolist()
tweet_docs = tweets.lemmas.tolist()

# Topic to document matrix (W) for LDA model
lda_W = lda_model.transform(tf)
# Word to topics matrix (H) for LDA model
lda_H = lda_model.components_


def display_topics(H, W, feature_names, orig_docs, n_words=15, n_docs=25):
    for i, topic in enumerate(H):
        print('Topic {}: '.format(i) + ' '.join([feature_names[word]
                                                 for word in topic.argsort()[: (-n_words - 1): -1]]))
        print()
        top_doc_ids = np.argsort(W[:, i])[:: -1][0: n_docs]
        for doc_id in top_doc_ids:
            print('Tweet:', repr(orig_docs[doc_id]))
        print('-' * 80)


display_topics(lda_H, lda_W, tf_names, tweets_orig, n_docs=0)

Topic 0: trump clinton year show life bill attack potus change donald ask job plan rally dem

--------------------------------------------------------------------------------
Topic 1: news tell people break police really play refugee everyone hear meet school wait name high

--------------------------------------------------------------------------------
Topic 2: think would call woman right need thank let pjnet way help bad ccot realdonaldtrump mean

--------------------------------------------------------------------------------
Topic 3: make trump politic tcot love state big may money usa face sign conservative bring government

--------------------------------------------------------------------------------
Topic 4: good black see trump video watch world first debate could start say must happen people

--------------------------------------------------------------------------------
Topic 5: go man never medium back pay ever trump tweet old many twitter girl believe tax

-----------

In [13]:
topic_names = ['Topic' + str(i) for i in range(len(lda_model.components_))]
doc_names = ['Doc' + str(i) for i in range(len(tweet_docs))]
word_names = ['Word ' + str(i) for i in range(len(tf_names))]

In [14]:
# Create df with the topic probabilities (cols) for each doc (rows)
doc_topic_df = pd.DataFrame(np.round(lda_W, 2), columns=topic_names, index=doc_names)

# Add column with dominant topic for each doc
doc_topic_df['dominant_topic'] = np.argmax(doc_topic_df.values, axis=1)
print(doc_topic_df.head(10))

       Topic0  Topic1  Topic2  Topic3  Topic4  Topic5  Topic6  Topic7  Topic8  \
Doc0     0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02   
Doc1     0.07    0.07    0.07    0.07    0.07    0.07    0.07    0.07    0.07   
Doc2     0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01   
Doc3     0.03    0.03    0.03    0.03    0.03    0.03    0.03    0.03    0.03   
Doc4     0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.68    0.01   
Doc5     0.17    0.01    0.01    0.60    0.01    0.01    0.01    0.01    0.01   
Doc6     0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.61    0.16   
Doc7     0.03    0.03    0.53    0.03    0.03    0.03    0.03    0.03    0.03   
Doc8     0.68    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01   
Doc9     0.03    0.03    0.03    0.03    0.53    0.03    0.03    0.03    0.03   
Doc10    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.77    0.02   
Doc11    0.01    0.01    0.0

In [15]:
# Create df with document topic distribution (num docs per topic)
topic_dist_df = doc_topic_df['dominant_topic'].value_counts().reset_index(name='Num docs')
topic_dist_df.columns = ['Topic_num', 'Num_docs']
topic_dist_df['Proportion'] = topic_dist_df.Num_docs.apply(lambda x: x / len(tweet_docs))
print()
print(topic_dist_df)


    Topic_num  Num_docs  Proportion
0           0     33299    0.164379
1           2     21810    0.107664
2           1     18944    0.093516
3           3     17015    0.083994
4           4     16211    0.080025
5           8     12821    0.063290
6           9     12168    0.060067
7           6     11281    0.055688
8           5     10857    0.053595
9           7      9963    0.049182
10         10      9570    0.047242
11         11      9312    0.045968
12         12      7546    0.037251
13         14      6396    0.031574
14         13      5381    0.026563


In [16]:
def top_keywords(feature_names, H):
    keywords = np.array(feature_names)
    topic_keywords = []
    for weights in H:
        topic_keywords_locs = (-weights).argsort()
        topic_keywords.append(keywords.take(topic_keywords_locs))

    return pd.DataFrame(topic_keywords)


# Create df with top words (cols) per topic (rows)
topic_words_df = top_keywords(tf_names, lda_H)
topic_words_df.columns = word_names
topic_words_df.index = topic_names
print(topic_words_df.iloc[:, :5])

           Word 0           Word 1    Word 2          Word 3    Word 4
Topic0      trump          clinton      year            show      life
Topic1       news             tell    people           break    police
Topic2      think            would      call           woman     right
Topic3       make            trump   politic            tcot      love
Topic4       good            black       see           trump     video
Topic5         go              man     never          medium      back
Topic6       know        president   clinton           leave      find
Topic7       want            trump      stop           great      time
Topic8        get              say       day           trump   country
Topic9      white  realdonaldtrump   blicqer             die      keep
Topic10      vote         american      maga           trump  midnight
Topic11     obama          america       gop  hillaryclinton      give
Topic12       new            trump  campaign            post      real
Topic1

In [18]:
doc_topic_df.to_csv('../scripts/topic_modeling_objects/topics_per_doc_LDA.csv', index=True)
topic_dist_df.to_csv('../scripts/topic_modeling_objects/docs_per_topic_LDA.csv', index=False)
topic_words_df.to_csv('../scripts/topic_modeling_objects/words_per_topic_LDA.csv', index=True)

In [5]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, tf, cv, mds='tsne')
panel

D:\jmelt\Documents\UNCC\ITIS4510\Russian_Tweets\venv\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
0      -79.586159  735.812012       1        1  7.529637
2      530.875549  372.021790       2        1  7.453647
9      250.989868  586.880432       3        1  7.378792
8      160.698135  -65.314659       4        1  7.316427
11     495.827393   26.663721       5        1  7.226811
3      -59.436409  423.964996       6        1  7.089555
4      226.612396  246.723358       7        1  7.031905
6     -172.836365 -111.555473       8        1  6.576050
1     -363.451538  208.240631       9        1  6.255124
10    -524.962402  -61.302818      10        1  6.225827
14    -401.711182  519.973145      11        1  6.177583
5     -317.285156 -388.780853      12        1  6.162971
7      -58.743275  144.067169      13        1  6.133565
12     388.016296 -303.370972      14        1  5.779927
13      45.037899 -386.081116      15        1  5.662181, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
895   Default   8790.000000         get   8790.000000  30.0000  30.0000
2262  Default  31217.000000       trump  31217.000000  29.0000  29.0000
913   Default   6454.000000          go   6454.000000  28.0000  28.0000
1508  Default   8451.000000       obama   8451.000000  27.0000  27.0000
1896  Default   8081.000000         say   8081.000000  26.0000  26.0000
1317  Default   6117.000000        make   6117.000000  25.0000  25.0000
1469  Default   5616.000000         new   5616.000000  24.0000  24.0000
405   Default   7665.000000     clinton   7665.000000  23.0000  23.0000
1470  Default   4904.000000        news   4904.000000  22.0000  22.0000
2334  Default   4700.000000        vote   4700.000000  21.0000  21.0000
920   Default   4914.000000        good   4914.000000  20.0000  20.0000
1205  Default   4462.000000        know   4462.000000  19.0000  19.0000
994   Default   5150.000000     hillary   5150.000000  18.0000  18.0000
81    Default   4038.000000    american   4038.000000  17.0000  17.0000
1638  Default   4233.000000     politic   4233.000000  16.0000  16.0000
2140  Default   3971.000000        take   3971.000000  15.0000  15.0000
79    Default   4135.000000     america   4135.000000  14.0000  14.0000
546   Default   4056.000000         day   4056.000000  13.0000  13.0000
1675  Default   3887.000000   president   3887.000000  12.0000  12.0000
2150  Default   3917.000000        tcot   3917.000000  11.0000  11.0000
234   Default   3853.000000       black   3853.000000  10.0000  10.0000
2348  Default   4578.000000        want   4578.000000   9.0000   9.0000
1926  Default   3762.000000         see   3762.000000   8.0000   8.0000
1326  Default   3548.000000         man   3548.000000   7.0000   7.0000
2192  Default   3746.000000       think   3746.000000   6.0000   6.0000
695   Default   3232.000000    election   3232.000000   5.0000   5.0000
1300  Default   3494.000000        love   3494.000000   4.0000   4.0000
2421  Default   3492.000000       would   3492.000000   3.0000   3.0000
2429  Default   3504.000000        year   3504.000000   2.0000   2.0000
310   Default   3485.000000        call   3485.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
695   Topic15   3231.585249    election   3232.516666   2.8711  -2.9487
424   Topic15   2772.958534        come   2773.889951   2.8710  -3.1018
1279  Topic15   2237.680703        live   2238.612121   2.8709  -3.3163
2273  Topic15   2187.324031         try   2188.255449   2.8709  -3.3390
2142  Topic15   1881.022826        talk   1881.954244   2.8709  -3.4899
1292  Topic15   1812.630988        lose   1813.562405   2.8708  -3.5269
1570  Topic15   1585.612844       party   1586.544262   2.8708  -3.6607
1763  Topic15   1370.497172        read   1371.428589   2.8707  -3.8065
2047  Topic15   1315.927218      spe

In [ ]:
# Construct k-means clusters
clusters = KMeans(n_clusters=15,
                  random_state=100).fit_predict(lda_W)
colors = ['#E53935', '#D81B60', '#8E24AA', '#5E35B1', '#3949AB',
          '#1976D2', '#0288D1', '#0097A7', '#00796B', '#8D6E63',
          '#689F38', '#AFB42B', '#FBC02D', '#90A4AE', '#F57C00']

# Build Singluar Value Decomposition (SVD) model
svd_model = TruncatedSVD(n_components=2)
lda_output_svd = svd_model.fit_transform(lda_W)

x = lda_output_svd[:, 0]
y = lda_output_svd[:, 1]

In [ ]:
fig = plt.figure(figsize=(12, 12))
for topic, color in zip(np.unique(clusters), colors):
    i = np.where(clusters == topic)
    plt.scatter(x[i], y[i], c=color, label=topic, alpha=0.2)

plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.title('Segregation of Topic Clusters')
leg = plt.legend(title='Topic', loc='best', ncol=3)

for lh in leg.legendHandles:
    lh.set_alpha(1)

fig.savefig('../visuals/LDA_topic_clusters.png')
plt.show()